# 가상사용자 생성

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

# 상태정의

In [7]:
from typing import TypedDict, Annotated
from langgraph.graph import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages] # 사용자 - 상담사 간의 대화 메시지

# 상담사, 고객 역할 정의

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langchain_core.output_parsers import StrOutputParser

def call_chatbot(messages: list[BaseMessage]) -> dict:
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system",
             "You are a customer support agent for an airline. Answer in Korean."),
             MessagesPlaceholder(variable_name="messages")
        ]
    )

    model = ChatOpenAI(model="gpt-4o-mini")
    chain = prompt | model | StrOutputParser()
    return chain.invoke({"messages": messages})


In [11]:
call_chatbot(["user", "안녕하세요?"])

'안녕하세요! 고객님, 어떻게 도와드릴까요?'

# 고객역할 에이전트 정의

In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

def create_scenario(name: str, instructions: str):
    system_prompt_template = """
    You are a customer of an airline conpany. \n
    You are interacting with a user who is a customer support person.
    
    Your name is {name}.
    
    # instructions:
    {instructions}
    
    [IMPORTANT]
    - When you are finished with the conversation, respond with a single word 'FINISHED'
    - You must speak in Korean.""    
    """
    
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt_template),
            MessagesPlaceholder(variable_name="messages")
        ]
    )

    prompt = prompt.partial(name=name, instructions=instructions)
    return prompt

In [13]:
# 사용자 지시사항을 정의합니다.
instructions = """You are trying to get a refund for the trip you took to Jeju Island. \
You want them to give you ALL the money back. This trip happened last year."""

# 사용자 이름을 정의합니다.
name = "Jaeho"

create_scenario(name, instructions).pretty_print()

================================ System Message ================================


    You are a customer of an airline conpany. 

    You are interacting with a user who is a customer support person.

    Your name is {name}.

    # instructions:
    {instructions}

    [IMPORTANT]
    - When you are finished with the conversation, respond with a single word 'FINISHED'
    - You must speak in Korean.""    
    

============================= Messages Placeholder =============================

{messages}


In [14]:
model = ChatOpenAI(model="gpt-4o-mini")
simulated_user = create_scenario(name, instructions) | model | StrOutputParser()

In [16]:
from langchain_core.messages import HumanMessage
# 시뮬레이션된 사용자에게 메시지를 전달(상담사 -> 고객)
messages = [HumanMessage(content="안녕하세요? 어떻게 도와드릴까요?")]
simulated_user.invoke({"messages": messages})

'안녕하세요. 작년 제주도 여행에 대한 환불을 요청하고 싶습니다. 전체 금액을 환불받고 싶습니다. 어떻게 진행해야 할까요?'

In [17]:
from langchain_core.messages import AIMessage

def ai_assistant_node(messages):

    ai_response = call_chatbot(messages)
    return {"messages": [("assistant", ai_response)]}

In [18]:
ai_assistant_node(
    [
        ("user", "안녕하세요?"),
        ("assistant", "안녕하세요! 어떻게 도와드릴까요?"),
        ("user", "환불 어떻게 진행하나요?")
    ]
)

{'messages': [('assistant',
   '환불을 진행하시려면 다음 절차를 따라 주시기 바랍니다:\n\n1. **예약 확인**: 예약한 항공편의 예약 번호와 개인 정보를 준비해 주세요.\n2. **고객 서비스 센터에 연락**: 전화 또는 이메일을 통해 저희 고객 서비스 센터에 연락해 주시면 됩니다.\n3. **환불 사유 설명**: 환불을 요청하시는 사유를 말씀해 주시면 보다 원활한 처리가 가능합니다.\n4. **환불 처리 안내**: 고객 서비스 팀이 환불 절차를 안내해 드리며, 필요한 경우 추가 정보를 요청할 수 있습니다.\n\n환불 처리에는 다소 시간이 소요될 수 있으며, 자세한 사항은 고객 서비스 팀에 문의하시면 더 정확한 정보를 얻으실 수 있습니다. 추가 질문이 있으시면 언제든지 말씀해 주세요!')]}